In [5]:
from utility import create_milvus_collection, drop_milvus_collection
from vector_db import MilvusCollection


create_milvus_collection()

collection = MilvusCollection()
collection.create_partition("default")

In [6]:
data = [
    [101, 102, 103, 104, 105],
    [[1.1, 1.2], [2.1, 2.2], [3.1, 3.2],[4.1, 4.2], [5.1, 5.2]]
]

collection.insert(data, "temp")

In [7]:
search_params = {
    "metric_type": "L2", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}

results = collection.search([[0.1, 0.2]], "word_vector", search_params, 2, partition_names=["temp"])
results

["['id: 101, distance: 2.0, entity: {}', 'id: 102, distance: 7.999999523162842, entity: {}']"]

In [8]:
collection.drop_partition("temp")
drop_milvus_collection()